In [ ]:
!pip install kaggle


In [ ]:
from google.colab import files
files.upload()  # use the dialog to select & upload kaggle.json



Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"kgurnoor0101","key":"f67794338d3651c679faa70d2ee15494"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!pip install requests


In [ ]:
!kaggle datasets download -d ashfakyeafi/road-vehicle-images-dataset


Dataset URL: https://www.kaggle.com/datasets/ashfakyeafi/road-vehicle-images-dataset
License(s): DbCL-1.0
road-vehicle-images-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -q road-vehicle-images-dataset.zip -d traffic_data/

replace traffic_data/trafic_data/data_1.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: [n]o
error:  invalid response [[n]o]
replace traffic_data/trafic_data/data_1.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace traffic_data/trafic_data/train/images/01_jpg.rf.8d8a2f0f90d5b83893cd252acd832c93.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
import random
import shutil
from google.colab import files

# Set random seed for reproducibility (optional)
random.seed(42)

# Source and destination paths
src_dir = "traffic_data/trafic_data/train/images"
train_dst = "content/images/train"
test_dst = "content/images/test"

# Reset destination folders (clean start)
shutil.rmtree(train_dst, ignore_errors=True)
shutil.rmtree(test_dst, ignore_errors=True)
os.makedirs(train_dst, exist_ok=True)
os.makedirs(test_dst, exist_ok=True)

# Get all image files (ignore non-images)
all_images = [f for f in os.listdir(src_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
random.shuffle(all_images)

# First select test set (50 images)
test_random = all_images[:50]
remaining = all_images[50:]

# From remaining, take 92 train images
train_random = remaining[:92]

# Copy random train images
for img in train_random:
    shutil.copy(os.path.join(src_dir, img), os.path.join(train_dst, img))

# Copy test images
for img in test_random:
    shutil.copy(os.path.join(src_dir, img), os.path.join(test_dst, img))

print("✅ Random dataset split done.")
print("Now please upload exactly 8 personal images...")

# Upload your 8 personal images
uploaded = files.upload()

if len(uploaded) != 8:
    print(f"⚠️ You uploaded {len(uploaded)} images. Expected exactly 8!")
else:
    for img_name in uploaded.keys():
        # Avoid overwriting if same filename exists
        dst_path = os.path.join(train_dst, img_name)
        if os.path.exists(dst_path):
            base, ext = os.path.splitext(img_name)
            dst_path = os.path.join(train_dst, f"{base}_personal{ext}")
        shutil.move(img_name, dst_path)

    print("✅ Added 8 personal images to train set.")

# Final counts
print(f"📂 Train set size: {len(os.listdir(train_dst))} (expected 100)")
print(f"📂 Test set size: {len(os.listdir(test_dst))} (expected 50)")


✅ Random dataset split done.
Now please upload exactly 8 personal images...


Saving IMG-20250911-WA0015.jpg to IMG-20250911-WA0015.jpg
Saving IMG-20250921-WA0020.jpg to IMG-20250921-WA0020.jpg
Saving IMG-20250921-WA0021.jpg to IMG-20250921-WA0021.jpg
Saving IMG-20250921-WA0022.jpg to IMG-20250921-WA0022.jpg
Saving IMG-20250921-WA0023.jpg to IMG-20250921-WA0023.jpg
Saving IMG-20250921-WA0024.jpg to IMG-20250921-WA0024.jpg
Saving IMG-20250921-WA0025.jpg to IMG-20250921-WA0025.jpg
Saving IMG-20250921-WA0026.jpg to IMG-20250921-WA0026.jpg
✅ Added 8 personal images to train set.
📂 Train set size: 100 (expected 100)
📂 Test set size: 50 (expected 50)


In [ ]:
import shutil

# Create a zip file of the folder
shutil.make_archive("images_dataset", 'zip', "/content/content/images")

# Download to local laptop
from google.colab import files
files.download("images_dataset.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ============================================
# Complete Train/Test + Labellerr Upload Workflow
# Using your personal dataset (already prepared)
# ============================================

!pip install -q https://github.com/tensormatics/SDKPython/releases/download/prod/labellerr_sdk-1.0.0.tar.gz
!git clone https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision.git || true

import sys, os, shutil
sys.path.append("/content/Hands-On-Learning-in-Computer-Vision/SDK Tutorials")

from labellerr.client import LabellerrClient
from labellerr.exceptions import LabellerrError

# 1️⃣ Set your credentials in Colab environment
client_id  = os.getenv("LABEL_CLIENT_ID")
api_key    = os.getenv("LABEL_API_KEY")
api_secret = os.getenv("LABEL_API_SECRET")
email      = "gurnoorkaur4oct24@gmail.com"

if not all([client_id, api_key, api_secret, email]):
    raise ValueError("❌ Set LABEL_CLIENT_ID, LABEL_API_KEY, LABEL_API_SECRET, LABEL_EMAIL in environment!")

# Initialize Labellerr client
client = LabellerrClient(api_key=api_key, api_secret=api_secret)

# ============================================
# 2️⃣ Use your already-prepared dataset
# ============================================

train_dst = "/content/content/images/train"   # contains 100 images (92 + 8 personal)
test_dst  = "/content/content/images/test"    # contains 50 images

print(f"📂 Train images: {len(os.listdir(train_dst))}")
print(f"📂 Test images: {len(os.listdir(test_dst))}")
# ============================================
# 3️⃣ Labellerr Annotation Guide
# ============================================
annotation_guide = [
    {
        "question_number": 1,
        "question": "Objects in Image",
        "question_id": "q-objects-001",
        "option_type": "polygon",
        "required": True,
        "options": [
            {"option_name": "vehicle"},
            {"option_name": "pedestrian"}
        ],
        "question_metadata": []
    }
]

# ============================================
# 4️⃣ Helper function to create projects
# ============================================

def create_project(payload, project_type=""):
    try:
        result = client.initiate_create_project(payload)
        proj_id = result['project_id']['response']['project_id']
        print(f"✅ {project_type} project created! Project ID: {proj_id}")
    except LabellerrError as e:
        print(f"❌ {project_type} project creation failed:", e)
    except Exception as ex:
        print(f"⚠️ Unexpected error for {project_type}:", ex)

# ============================================
# 5️⃣ Create Train and Test projects (upload personal dataset)
# ============================================

train_payload = {
    'client_id': client_id,
    'dataset_name': 'vehicles_trainset-current',
    'dataset_description': 'Train set (includes personal images) for YOLO-Seg annotation',
    'data_type': 'image',
    'created_by': email,
    'project_name': 'Vehicle_Segmentation_Train-current',
    'annotation_guide': annotation_guide,
    'rotation_config': {
        'annotation_rotation_count': 1,
        'review_rotation_count': 1,
        'client_review_rotation_count': 1
    },
    'autolabel': False,
    'folder_to_upload': train_dst
}

test_payload = {
    'client_id': client_id,
    'dataset_name': 'vehicles_testset-current',
    'dataset_description': 'Test set for YOLO-Seg',
    'data_type': 'image',
    'created_by': email,
    'project_name': 'Vehicle_Segmentation_Test-current',
    'annotation_guide': annotation_guide,
    'rotation_config': {
        'annotation_rotation_count': 1,
        'review_rotation_count': 1,
        'client_review_rotation_count': 1
    },
    'autolabel': False,
    'folder_to_upload': test_dst
}

create_project(train_payload, "Train")
create_project(test_payload, "Test")


  Preparing metadata (setup.py) ... done
fatal: destination path 'Hands-On-Learning-in-Computer-Vision' already exists and is not an empty directory.
📂 Train images: 100
📂 Test images: 50
Rotation configuration validated . . .
Creating dataset . . .
Total file count: 100
Total file size: 5.1 MB
CPU count 2  Batch Count 1


Dataset created and ready for use
Annotation guidelines created
✅ Train project created! Project ID: fancy_shivering_magpie_38178
Rotation configuration validated . . .
Creating dataset . . .
Total file count: 50
Total file size: 1.9 MB
CPU count 2  Batch Count 1


Dataset created and ready for use
Annotation guidelines created
✅ Test project created! Project ID: odelle_moderate_whippet_90705
